#Main App - Using Streamlit & LangChain

Before starting , we need the following python codes in the google drive link given below , uploaded into the root directory of Google Colab Runtime. It is not the case when it comes to running it locally , since Colab has its files reseted for each runtime.

[Google Drive](https://drive.google.com/drive/folders/12bc8tO4WWYDariroqmi2dotzWLhsa9yt?usp=sharing)


Installing all required packages.

In [1]:
!pip -q install langchain huggingface_hub tiktoken
!pip -q install chromadb streamlit
!pip -q install PyPDF2 pypdf sentence_transformers
!pip -q install --upgrade together
!pip -q install -U FlagEmbedding
!npm install localtunnel
!pip install -q pypdf python-dotenv transformers einops accelerate bitsandbytes sentence_transformers langchain llama-index streamlit torch mysql-connector-python pymysql
!pip install -r requirements.txt
!pip install auto-gptq torch huggingface_hub transformers click
!pip install jsonschema-specifications==2023.7.1
!pip install auto-gptq optimum protobuf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 448.1/448.1 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 48.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.3/66.3 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 69.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 84.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.0/301.0 kB 5.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for optimum: filename=optimum-1.13.2-py3-none-any.whl size=395599 sha256=a73e252f840654ab24165722990809a003d8466704f310b68c3278cd6c3f2b48
  Stored in directory: /root/.cache/pip/wheels/6e/b7/2c/79405d98f0943373d8546daeae25a3d377f7659ca0cbe48699
Successfully built optimum


In [2]:
!pip show langchain

Name: langchain
Version: 0.0.267
Summary: Building applications with LLMs through composability
Home-page: https://www.github.com/hwchase17/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, async-timeout, dataclasses-json, langsmith, numexpr, numpy, openapi-schema-pydantic, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: llama-index


#Choosing Desired Model

One can choose any model for LLMs and for Embedding.
The value for these models are constants and can be changed from the constants.py file that was downloaded earlier. It gives you a breif idea for the choice of model that fits best for your hardware and usage.

In [3]:
!pip install --upgrade auto-gptq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 11.2 MB/s eta 0:00:00
  Attempting uninstall: auto-gptq
    Found existing installation: auto-gptq 0.2.2
    Uninstalling auto-gptq-0.2.2:
      Successfully uninstalled auto-gptq-0.2.2


# Creating an Streamlit App to do a Q&A Chat


In [4]:
%%writefile mainapp.py

##Importing all required packages.

import torch
import together
import subprocess
import os
import streamlit as st
from langchain.chains import RetrievalQA
import logging
from langchain.vectorstores import Chroma
from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceInstructEmbeddings,HuggingFaceEmbeddings
from langchain.prompts import PromptTemplate
from typing import Any, Dict, List, Mapping, Optional
from pydantic import Extra, Field, root_validator
from langchain.callbacks.manager import CallbackManagerForLLMRun
from langchain.llms.base import LLM
from langchain.llms.utils import enforce_stop_tokens
from langchain.utils import get_from_dict_or_env
import textwrap
from langchain.memory import ConversationBufferMemory
from transformers import (
    GenerationConfig,
    pipeline
)

##Importing values from the python files that was loaded.

from load_models import (
    load_quantized_model_gguf_ggml,
    load_quantized_model_qptq,
    load_full_model,
)
from constants import (
    CHROMA_SETTINGS,
    EMBEDDING_MODEL_NAME,
    PERSIST_DIRECTORY,
    MAX_NEW_TOKENS,
    MODELS_PATH,
    MODEL_ID,
    MODEL_BASENAME
)

##Defining a function for loading a model.

def load_model(device_type, model_id, model_basename=None, LOGGING=logging):
    """
    Select a model for text generation using the HuggingFace library.
    If you are running this for the first time, it will download a model for you.
    subsequent runs will use the model from the disk.

    Args:
        device_type (str): Type of device to use, e.g., "cuda" for GPU or "cpu" for CPU.
        model_id (str): Identifier of the model to load from HuggingFace's model hub.
        model_basename (str, optional): Basename of the model if using quantized models.
            Defaults to None.

    Returns:
        HuggingFacePipeline: A pipeline object for text generation using the loaded model.

    Raises:
        ValueError: If an unsupported model or device type is provided.
    """
    logging.info(f"Loading Model: {model_id}, on: {device_type}")
    logging.info("This action can take a few minutes!")

    if model_basename is not None:
        if ".gguf" in model_basename.lower():
            llm = load_quantized_model_gguf_ggml(model_id, model_basename, device_type, LOGGING)
            return llm
        elif ".ggml" in model_basename.lower():
            model, tokenizer = load_quantized_model_gguf_ggml(model_id, model_basename, device_type, LOGGING)
        else:
            model, tokenizer = load_quantized_model_qptq(model_id, model_basename, device_type, LOGGING)
    else:
        model, tokenizer = load_full_model(model_id, model_basename, device_type, LOGGING)

    # Load configuration from the model to avoid warnings
    generation_config = GenerationConfig.from_pretrained(model_id)

    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_length=MAX_NEW_TOKENS,
        temperature=0.2,
        # top_p=0.95,
        repetition_penalty=1.15,
        generation_config=generation_config,
    )

    local_llm = HuggingFacePipeline(pipeline=pipe)
    logging.info("Local LLM Loaded")

    return local_llm



def model_memory():
    # Adding history to the model.
    template = """Use the following pieces of context to answer the question at the end. If you don't know the answer,\
    just say that you don't know, don't try to make up an answer.

    {context}

    {history}
    Question: {question}
    Helpful Answer:"""

    prompt = PromptTemplate(input_variables=["history", "context", "question"], template=template)
    memory = ConversationBufferMemory(input_key="question", memory_key="history")

    return prompt, memory

# A function for uploading documents per user's choice.

def upload_documents():
    uploaded_file = st.file_uploader("Upload a document", type=["txt", "pdf", "docx"])
    if st.button("Submit"):
        if uploaded_file:
            # Create a directory to store uploaded documents if it doesn't exist
            if not os.path.exists("SOURCE_DOCUMENTS"):
                os.makedirs("SOURCE_DOCUMENTS")

            # Store the uploaded document
            with open(os.path.join("SOURCE_DOCUMENTS", uploaded_file.name), "wb") as f:
                f.write(uploaded_file.read())
                run_langest_commands = ["python", "ingest.py"]
                run_langest_commands.append("--device_type")
                run_langest_commands.append(DEVICE_TYPE)
                result = subprocess.run(run_langest_commands, capture_output=True)
                st.session_state.result = result
            st.success("Document uploaded successfully!")



# Sidebar contents
with st.sidebar:
    st.title("💬Talk with your data💬")
    st.markdown(
        """
    ## About
    This app is an LLM-powered chatbot built using Streamlit and Langchain.
      """
    )



DEVICE_TYPE = "cuda"

# session_state is used here to make sure streamlit doesn't reload entirely once the last line of the app is reached.
# It makes sure it keeps the session online till the user decides to close it. It seems to be a bug in streamlit.

# Define the retriever
# load the vectorstore
if "EMBEDDINGS" not in st.session_state:
    EMBEDDINGS = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL_NAME)
    st.session_state.EMBEDDINGS = EMBEDDINGS

if "DB" not in st.session_state:
    DB = Chroma(
        persist_directory=PERSIST_DIRECTORY,
        embedding_function=st.session_state.EMBEDDINGS,
        client_settings=CHROMA_SETTINGS,
    )
    st.session_state.DB = DB

if "RETRIEVER" not in st.session_state:
    RETRIEVER = DB.as_retriever()
    st.session_state.RETRIEVER = RETRIEVER

if "LLM" not in st.session_state:
    LLM = load_model(device_type=DEVICE_TYPE, model_id=MODEL_ID)
    st.session_state["LLM"] = LLM

if "QA" not in st.session_state:
    prompt, memory = model_memory()

    QA = RetrievalQA.from_chain_type(
        llm=LLM,
        chain_type="stuff",
        retriever=RETRIEVER,
        return_source_documents=True,
        chain_type_kwargs={"prompt": prompt, "memory": memory},
    )
    st.session_state["QA"] = QA

if "result" not in st.session_state:
  run_langest_commands = ["python", "ingest.py"]
  run_langest_commands.append("--device_type")
  run_langest_commands.append(DEVICE_TYPE)
  result = subprocess.run(run_langest_commands, capture_output=True)
  st.session_state.result = result

# Create a sidebar navigation
page = st.sidebar.selectbox("Select the interface", ["Upload Documents", "Q&A Chat"])

if page == "Upload Documents":
    st.title("Upload Documents")
    upload_documents()

elif page == "Q&A Chat":
    st.title("Q&A Retrieval App 💬")

    # Create a text input box for the user


    prompt = st.text_input("Input your prompt here")
    # If the user hits enter
    if prompt:
        # Then pass the prompt to the LLM
        response = st.session_state["QA"](prompt)
        answer, docs = response["result"], response["source_documents"]
        # ...and write it out to the screen
        st.write("> "+answer)

        # With a streamlit expander
        with st.expander("Document Similarity Search"):
            # Find the relevant pages
            search = st.session_state.DB.similarity_search_with_score(prompt)
            # Write out the first
            #for i, doc in enumerate(search):
                # print(doc)
                #st.write(f"Source Document # {i + 1} : {doc[0].metadata['source'].split('/')[-1]}")
                #st.write(doc[0].page_content)
                #st.write("--------------------------------")

Writing mainapp.py


#Running this app in Google Colab
( since streamlit required a terminal for running the app properly )

In [8]:
!streamlit run /content/mainapp.py &>/content/logs_main.txt &

Once run , a new log file is saved in the root directory in the name logs_main.txt. Take a note of the external url's IP ( needn't include the port number or http ).

In [11]:
!npx localtunnel --port 8501

npx: installed 22 in 1.315s
your url is: https://bumpy-tips-dig.loca.lt


Once run , the above command will generate a localtunnel. Enter the noted IP in the textbox and enter and wait for the website to load.
Then , you can start chatting with your documents.

#Future Scope

Implementation of a DB to store files using MySQL Database

---
Note that these are future scopes of this app and hasn't been completed yet.


In [ ]:
!pip install mysql-connector-python

**MySQL Tables - in DataBase**


---
Here are the commands to create the database and the tables.


In [ ]:
#Creating the DataBase
CREATE DATABASE your_database;
CREATE USER 'your_user'@'localhost' IDENTIFIED BY 'your_password';
GRANT ALL PRIVILEGES ON your_database.* TO 'your_user'@'localhost';
FLUSH PRIVILEGES;

In [ ]:
#Uploaded Files Table
USE your_database;
CREATE TABLE uploaded_files (
    id INT AUTO_INCREMENT PRIMARY KEY,
    document_name VARCHAR(255),
    document_data LONGBLOB,
    document_domain VARCHAR(255)
);

In [ ]:
CREATE TABLE domain_options(
    id INT AUTO_INCREMENT PRIMARY KEY,
    domain_name VARCHAR(255),
    is_selected BOOLEAN
)

Let domain_options and uploaded documents be two tables in the database. The domain_options table has all the valid domains that can be accepted. The user can choose their domain based on the list that is fetched from the table.

In [ ]:
import mysql.connector

# Function to fetch available domain options
def get_domain_options():
    connection = mysql.connector.connect(
        host="your_mysql_host",
        user="your_mysql_user",
        password="your_mysql_password",
        database="your_database"
    )
    cursor = connection.cursor()

    cursor.execute("SELECT domain_name, is_selected FROM domain_options")
    options = cursor.fetchall()

    cursor.close()
    connection.close()

    return options

# Function to upload documents to MySQL
def upload_documents():
    domain_options = get_domain_options()
    selected_domain = None

    if st.button("Submit"):
        if uploaded_file:
            # Read the file content
            file_content = uploaded_file.read()

            # Prompt the user to select a domain
            st.text("Select a domain:")
            domain_names = [domain_name for domain_name, is_selected in domain_options]
            selected_domain = st.selectbox("Domain", domain_names)

            # Get the is_selected status of the selected domain
            for domain_name, is_selected in domain_options:
                if domain_name == selected_domain:
                    is_selected_status = is_selected

            # Insert the document into the MySQL database
            insert_query = "INSERT INTO uploaded_documents (document_name, document_data, document_domain) VALUES (%s, %s, %s)"
            values = (uploaded_file.name, file_content, selected_domain)
            cursor.execute(insert_query, values)
            connection.commit()

            # Close the cursor and connection
            cursor.close()
            connection.close()

            st.success("Document uploaded successfully!")

    # Rest of your code remains unchanged

#####################################
#Rest of your code remains unchanged#
#####################################

Now , for ingestion process, ingesting all the documents takes a lot of time. Therefore, we can give the user the choice to ingest document from a particular domain only so that , it is much more effective.

In [ ]:
import mysql.connector

# Function to fetch existing domains from the uploaded_documents table
def get_existing_domains():
    connection = mysql.connector.connect(
        host="your_mysql_host",
        user="your_mysql_user",
        password="your_mysql_password",
        database="docs"
    )
    cursor = connection.cursor()

    cursor.execute("SELECT DISTINCT document_domain FROM uploaded_documents")
    existing_domains = [row[0] for row in cursor.fetchall()]

    cursor.close()
    connection.close()

    return existing_domains

# Function for document ingestion
def ingest_documents(selected_domain):
    # Fetch documents based on the selected domain
    connection = mysql.connector.connect(
        host="your_mysql_host",
        user="your_mysql_user",
        password="your_mysql_password",
        database="docs"
    )
    cursor = connection.cursor()

    select_query = "SELECT document_name, document_data FROM uploaded_documents WHERE document_domain = %s"
    cursor.execute(select_query, (selected_domain,))
    documents = cursor.fetchall()

    # Process and ingest the selected documents
    for doc_name, doc_data in documents:
        # Ingestion of the document here
        # .... ( same code as that of mainapp.py )

    cursor.close()
    connection.close()

# Function to present the domain selection to the user
def select_domain_for_ingestion():
    existing_domains = get_existing_domains()

    if existing_domains:
        st.text("Select a domain for ingestion:")
        selected_domain = st.selectbox("Domain", existing_domains)
        if st.button("Start Ingestion"):
            ingest_documents(selected_domain)
            st.success(f"Ingestion for the '{selected_domain}' domain has started.")
    else:
        st.warning("No documents available for ingestion.")

# Rest of your code remains unchanged
